Initial Data Observation

In [13]:
#QUESTIONS WE WANT TO ANSWER:

# Determine which aircraft are the lowest risk for the company to start this new business endeavor
# They are interested in purchasing and operating airplanes for commercial and private enterprises
    #1ST RECC. PRIVATE
    #2ND RECC. PRIVATE
# Actionable insights that the head of the new aviation division can use to help decide which aircraft to purchase.

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
df = pd.read_csv('../data/archive/AviationData.csv', encoding='cp1252', low_memory=False)
uscodes_df = pd.read_csv('../data/archive/USState_Codes.csv', encoding='cp1252', low_memory=False)

In [16]:
df.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [17]:
# Replace'.' with ' '
df.columns = [c.replace('.', ' ') for c in df]

#We are only concerned with Airplanes in the Aircraft Category
df = df.loc[df['Aircraft Category'] == 'Airplane']

#Standardizing all of the airplane makes by capitalizing.
df['Make'] = df['Make'].str.upper()
df['Air carrier'] = df['Air carrier'].str.title()

df['Weather Condition'] = df['Weather Condition'].str.upper()

#Dropping rows that are 'Amateur Built == 'Yes''
df.loc[df['Amateur Built'] == 'No']

#Dropping columns that are not relevant
df = df.drop(columns=['Amateur Built', 'Injury Severity', 'Weather Condition', 'Publication Date', 'Aircraft Category', 'Latitude', 'Longitude', 
                                        'Airport Code', 'Airport Name', 'Registration Number', 'Event Id', 'Air carrier', 'Schedule', 'Broad phase of flight'])

df['FAR Description'].replace(to_replace= '121', value='Part 121', inplace=True)
df['FAR Description'].replace(to_replace= '135', value='Part 135', inplace=True)
df['FAR Description'].replace(to_replace= '137', value='Part 137', inplace=True)
df['FAR Description'].replace(to_replace= '129', value='Part 129', inplace=True)
df['FAR Description'].replace(to_replace= 'Part 137', value='Part 137: Agricultural', inplace=True)
df['FAR Description'].replace(to_replace= 'Part 135', value='Part 135: Air Taxi & Commuter', inplace=True)
df['FAR Description'].replace(to_replace= 'Part 121', value='Part 121: Air Carrier', inplace=True)
df['FAR Description'].replace(to_replace= 'Part 129', value='Part 129: Foreign', inplace=True)
df['FAR Description'].replace(to_replace= 'Part 129', value='Part 129: Foreign', inplace=True)
df['FAR Description'].replace(to_replace= 'NUSN', value='Non-U.S., Non-Commercial', inplace=True)
df['FAR Description'].replace(to_replace= 'NUSC', value='Non-U.S., Commercial', inplace=True)
df['FAR Description'].replace(to_replace= '091', value='Part 91: General Aviation', inplace=True)
df['FAR Description'].replace(to_replace= '125', value='Part 125: 20+ Pax,6000+ lbs', inplace=True)
df['FAR Description'].replace(to_replace= 'UNK', value='Unknown', inplace=True)

#Keep 'Part 91: General Aviation', 'Non-U.S., Commercial', 'Part 121: Air Carrier', 'Part 129: Foreign', 'Unknown' 
df = df[(df['FAR Description']=='Part 91: General Aviation') | (df['FAR Description']=='Non-U.S., Commercial') |
  (df['FAR Description']=='Part 121: Air Carrier') | (df['FAR Description']=='Part 129: Foreign') |
  (df['FAR Description']=='Unknown')]                 

#Dropping all rows before 1990 - Average lifespan for a plane is 30 years                
df = df[(df['Event Date'] > '1990-00-00')]

df
                

,Investigation Type,Accident Number,Event Date,Location,Country,Aircraft damage,Make,Model,Number of Engines,Engine Type,FAR Description,Purpose of flight,Total Fatal Injuries,Total Serious Injuries,Total Minor Injuries,Total Uninjured,Report Status
24818,Accident,NYC90LA058,1990-01-28,"ROXBURY, ME",United States,Destroyed,TAYLORCRAFT,BC12D,1.0,Unknown,Part 91: General Aviation,Personal,NaN,NaN,NaN,2.0,Probable Cause
24820,Accident,NYC90LA059,1990-01-28,"PITTSTOWN, NJ",United States,Substantial,GLOBE,SWIFT GCIB,1.0,Reciprocating,Part 91: General Aviation,Personal,NaN,NaN,NaN,1.0,Probable Cause
24841,Accident,NYC90FA063,1990-02-01,"FAIRFIELD, CT",United States,Destroyed,PIPER,PA-28-181,1.0,Reciprocating,Part 91: General Aviation,Personal,1.0,NaN,NaN,NaN,Probable Cause
25079,Accident,ANC90LA043,1990-03-16,"SULATNA CROSS, AK",United States,Substantial,CESSNA,170B,1.0,Reciprocating,Part 91: General Aviation,Personal,NaN,NaN,NaN,2.0,Probable Cause
25228,Accident,ATL90FA095B,1990-04-09,"GADSDEN, AL",United States,Destroyed,CESSNA,172,1.0,Reciprocating,Part 91: General Aviation,Unknown,2.0,NaN,NaN,7.0,Probable Cause
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88865,Accident,ERA23LA085,2022-12-12,"Knoxville, TN",United States,Substantial,CESSNA,172,1.0,NaN,Part 91: General Aviation,Instructional,0.0,0.0,0.0,1.0,NaN
88869,Accident,WPR23LA065,2022-12-13,"Lewistown, MT",United States,Substantial,PIPER,PA42,2.0,NaN,"Non-U.S., Commercial",NaN,0.0,0.0,0.0,1.0,NaN
88873,Accident,ERA23LA090,2022-12-14,"San Juan, PR",United States,Substantial,CIRRUS DESIGN CORP,SR22,1.0,NaN,Part 91: General Aviation,Personal,0.0,0.0,0.0,1.0,NaN
88877,Accident,ERA23LA091,2022-12-16,"Brooksville, FL",United States,Substantial,CESSNA,R172K,1.0,NaN,Part 91: General Aviation,Personal,0.0,1.0,0.0,0.0,NaN


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20973 entries, 24818 to 88886
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Investigation Type      20973 non-null  object 
 1   Accident Number         20973 non-null  object 
 2   Event Date              20973 non-null  object 
 3   Location                20967 non-null  object 
 4   Country                 20973 non-null  object 
 5   Aircraft damage         20031 non-null  object 
 6   Make                    20969 non-null  object 
 7   Model                   20960 non-null  object 
 8   Number of Engines       19394 non-null  float64
 9   Engine Type             18117 non-null  object 
 10  FAR Description         20973 non-null  object 
 11  Purpose of flight       18920 non-null  object 
 12  Total Fatal Injuries    18106 non-null  float64
 13  Total Serious Injuries  18102 non-null  float64
 14  Total Minor Injuries    18426 non-

In [21]:
df[['Make','Total Fatal Injuries', 'Total Serious Injuries']]

,Make,Total Fatal Injuries,Total Serious Injuries
24818,TAYLORCRAFT,NaN,NaN
24820,GLOBE,NaN,NaN
24841,PIPER,1.0,NaN
25079,CESSNA,NaN,NaN
25228,CESSNA,2.0,NaN
...,...,...,...
88865,CESSNA,0.0,0.0
88869,PIPER,0.0,0.0
88873,CIRRUS DESIGN CORP,0.0,0.0
88877,CESSNA,0.0,1.0


In [ ]:
#Merge double values
#aviation_df.replace(to_replace=, value= )

In [ ]:
#'Engine Type' column - nothing to clean yet
#aviation_df['Engine Type'].value_counts()

In [ ]:
#aviation_df['Schedule'].value_counts()
#SCHD    1497
#NSCH    1237
#UNK      256

In [ ]:
#Get rid of a lot in this column
#df['Purpose of flight'].value_counts()

In [ ]:
#len(aviation_df['Air carrier'].unique())

In [ ]:
#THIS CATEGORY MAYBE SHOULD NOT BE CONSIDERED RELEVANT - want airplane to operate in all weather conditions
#aviation_df['Weather Condition'].value_counts()

In [ ]:
#THIS CATEGORY MAYBE SHOULD NOT BE CONSIDERED RELEVANT - 
#the phase of the flgiht when the incident/accident occured should not matter
#aviation_df['Broad phase of flight'].value_counts()

In [ ]:
#THIS CATEGORY MAYBE SHOULD NOT BE CONSIDERED RELEVANT - the written report of the incident
#aviation_df['Air carrier'].value_counts()

In [ ]:
#aviation_df['Event Date'].value_counts()